<a href="https://colab.research.google.com/github/thopiyawan/topic-modeling-thai/blob/main/Copy_of_topic_modeling_th_result_performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pwd

/content


In [3]:
cd /content/drive/My Drive/SWU-Master-Thesis-63/Ploy/topicmodeling_thai

/content/drive/.shortcut-targets-by-id/1UHDKxOFjfONnMYU-auICT39cUoR5NF0D/SWU-Master-Thesis-63/Ploy/topicmodeling_thai


In [4]:
import time
start_time = time.time()

In [5]:
import requests
from bs4 import BeautifulSoup
from time import  time, sleep
from random import randint

# from googletrans import Translator

from tqdm import tqdm
from tqdm import tqdm_notebook
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from nltk import sent_tokenize, word_tokenize
from nltk.sentiment.vader import  SentimentIntensityAnalyzer
from nltk.sentiment.util import mark_negation

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [6]:
dataset = pd.read_csv("dataset/QA_pregnancy.csv",encoding='utf8')
print(dataset.shape)
dataset.head()

(9987, 5)


,question,href,date_ask,user,answer
0,ตั้งครรภ์,https://hd.co.th//ask/27734160,"October 01, 2019 17:30",\nHonestDocs User 1082087\n,เราได้รับคำถามของคุณแล้ว รอทีมแพทย์มาตอบคำถามส...
1,ตั้งครรภ์5เดือนสามารถตัดปากมดได้ไหม,https://hd.co.th//ask/51006398,"June 30, 2018 14:49",\nHonestDocs User 912491\n,NaN
2,ตั้งครรภ์14สัปดาค่ะปวดท้องเหนือสะดือเเนวยาวร่อ...,https://hd.co.th//ask/4362714,"August 31, 2018 15:07",\nHonestDocs User 929536\n,NaN
3,ตั้งครรภ์24สัปดาห์โดนหมากัดที่ขามีรอยถลอกต้องไ...,https://hd.co.th//ask/33971594,"November 07, 2018 18:58",\nHonestDocs User 954767\n,วัคซีนพิษสุนัทบ้าและอิมมูโนโกลบูลินสามารถให้ใน...
4,ตั้งครรภ์ แต่ได๋อัลตราซาวด์ดูหมอบอกมีเลือกออกบ...,https://hd.co.th//ask/48478898,"August 06, 2018 1:35",\nHonestDocs User 921834\n,รบกวนขอข้อมูลเพพิ่มเติมด้วยครับ คุณแม่อายุเท่า...


In [7]:
print(dataset.shape)
print("===============================================")
print(dataset.info())

(9987, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9987 entries, 0 to 9986
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  9987 non-null   object
 1   href      9987 non-null   object
 2   date_ask  9982 non-null   object
 3   user      9986 non-null   object
 4   answer    9529 non-null   object
dtypes: object(5)
memory usage: 390.2+ KB
None


In [8]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [9]:
try:
    %tensorflow_version 2.x
except:
    pass

In [10]:
#dev version
# !pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip

# release version 
! pip install pythainlp

!pip install epitran
!pip install sklearn_crfsuite
!pip install tensorflow deepcut
!pip install attacut

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
import pythainlp

pythainlp.__version__

'2.2.6'

In [13]:
dataset_q = dataset.iloc[0:50,0]

In [14]:
import re, string, unicodedata

In [15]:
from pythainlp import sent_tokenize, word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.util import normalize
from pythainlp.spell import correct


stopwords = list(thai_stopwords())


In [16]:
def preprocess_review(reviews): 
    cleaned_words = []
    for review in reviews:
        review = re.sub("[^\u0E00-\u0E7Fa-zA-Z0-9' ]|^'|'$|''", "", str(review))
        words = word_tokenize(review, engine="deepcut")
        tokenized_words = [word for word in words if word not in stopwords]
        # correct_word = correct(str(tokenized_words))
        # lemmatized_word = lemmatize_word(tokenized_words)
        cleaned_words.append(tokenized_words)
    return cleaned_words


In [17]:

# data_np = np.array(str(data_stopw))
data_stopw = preprocess_review(dataset_q)
data_stopw


[['ครรภ์'],
 ['ครรภ์', '5', 'เดือน', 'ตัด', 'ปาก', 'มด', 'ไหม'],
 ['ครรภ์',
  '14',
  'สัปดา',
  'ปวด',
  'ท้อง',
  'เหนือ',
  'สะดือ',
  'เเนวยาว',
  'ร่อง',
  'นม',
  'ทราบ',
  'เปน',
  'ไคำ',
  'ถาม',
  'ตอบ',
  'พยาบาล',
  'เชี่ยวชาญ'],
 ['ครรภ์',
  '24',
  'สัปดาห์',
  'โดน',
  'หมา',
  'กัด',
  'ขา',
  'รอย',
  'ถลอก',
  'หา',
  'หมอ',
  'ไหม'],
 ['ครรภ์',
  ' ',
  'ได๋',
  'อัลตราซาวด์',
  'ดู',
  'หมอ',
  'เลือก',
  'บริเวณ',
  'รก',
  ' ',
  'เรย'],
 ['ครรภ์',
  '5',
  'เดือน',
  'คะมี',
  'ตก',
  'ขาว',
  'สี',
  'น้ำตาล',
  'ขุ่น',
  'แดง',
  'เข้ม',
  'แห้ง',
  'กลิ่น',
  'อาการ',
  'ปกติ',
  'ทราบ',
  'อันตราย',
  'ไหม',
  'ค้ะ',
  'สาเหตุ',
  'ถาม',
  'ตอบ',
  'พยาบาล',
  'เชี่ยวชาญ'],
 ['ครรภ์', '3', 'เดือน', 'ทาน', 'ถาม', 'ตอบ', 'พยาบาล', 'เชี่ยวชาญ'],
 ['ตรวจ',
  'เลือด',
  'ประจำ',
  'ปี',
  'ทำ',
  'งาน',
  ' ',
  'แจ้ง',
  'เลือด',
  ' ',
  'ครรภ์',
  'ถาม',
  'ตอบ',
  'พยาบาล',
  'เชี่ยวชาญ'],
 ['ครรภ์',
  '8',
  'เดือน',
  'ปวด',
  'ฟัน',
  'กราม',
  'ซี่',
  'อุด

In [18]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
from multiprocessing import Pool
import nltk
nltk.download('wordnet')

# def lemmatize_stemming(text):
#     return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
#     def preprocess(text):
#       result = []
#     for token in gensim.utils.simple_preprocess(text):
#         if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
#             result.append(lemmatize_stemming(token))
#     return result

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
dataset = dataset[['question','user']]
dataset['index'] = dataset.index
dataset 

,question,user,index
0,ตั้งครรภ์,\nHonestDocs User 1082087\n,0
1,ตั้งครรภ์5เดือนสามารถตัดปากมดได้ไหม,\nHonestDocs User 912491\n,1
2,ตั้งครรภ์14สัปดาค่ะปวดท้องเหนือสะดือเเนวยาวร่อ...,\nHonestDocs User 929536\n,2
3,ตั้งครรภ์24สัปดาห์โดนหมากัดที่ขามีรอยถลอกต้องไ...,\nHonestDocs User 954767\n,3
4,ตั้งครรภ์ แต่ได๋อัลตราซาวด์ดูหมอบอกมีเลือกออกบ...,\nHonestDocs User 921834\n,4
...,...,...,...
9982,คุณหมอคะ หลังจากที่เรามีเพศสัมพันธ์ กี่วันคะ จ...,\nHonestDocs User 904412\n,9982
9983,ประจำเดือนขาดมาครั้งสุดท้ายเมื่อต้นเมษายนค่ะ แ...,\nHonestDocs User 904485\n,9983
9984,รับทำแท้งมั้ยค่ะ,\nHonestDocs User 907823\n,9984
9985,เดือนพคปจดมาช่วงต้นเดือนและกลางเดือน2รอบค่ะจำว...,\nHonestDocs User 906992\n,9985


In [20]:
from sklearn import preprocessing

doc_sample = dataset[dataset['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
      words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(simple_preprocess(doc_sample))

original document: 
['หมอค่ะ', 'คือหนูมีอะไรกับแฟนแล้ว1ครั้ง', 'และอีกวันต่อมามีอะไรกันอีกครั้ง', 'แฟนหลั่งนอกไม่ทันค่ะแต่ถึงกับสุดนะค่ะ', 'และมาหลั่งข้างนอกอีกทีน้ำอุสจิก้มีมากอยู่นะค่ะ', 'มีอะไรติด2วันอ้ะค่ะ', 'วันที่2เชื้ออุสจิจะแข้งแรงพอที่จะท้องได้มั้ยคาะ']


 tokenized and lemmatized document: 
['หมอค', 'อหน', 'อะไรก', 'บแฟนแล', 'คร', 'และอ', 'กว', 'นต', 'อมาม', 'อะไรก', 'นอ', 'กคร', 'แฟนหล', 'งนอกไม', 'นค', 'ะแต', 'งก', 'บส', 'ดนะค', 'และมาหล', 'งข', 'างนอกอ', 'กท', 'ำอ', 'สจ', 'มากอย', 'นะค', 'อะไรต', 'นอ', 'ะค', 'นท', 'เช', 'ออ', 'สจ', 'จะแข', 'งแรงพอท', 'จะท', 'องได', 'ยคาะ']


In [21]:
!pip install pathos

In [22]:

import pandas as pd
import numpy as np
import time
from pymongo import MongoClient
from bson.objectid import ObjectId

import datetime
import calendar
import scipy.stats
from datetime import timedelta, date
import pytz
from pytz import timezone
import multiprocessing
from multiprocessing import cpu_count #For Parallel


cores = cpu_count() #Number of CPU cores on your system
partitions = cores #Define as many partitions as you want to run parallel

# def lemmatize_stemming(text):
#     return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def parallelize(data, func):
    data_split = np.array_split(data, partitions)
    pool = multiprocessing.Pool(cores)     #สร้าง multiprocessor จำนวนเท่ากับ core cpu ของ server
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

#ตัดคำ
def preprocess(documents):
    # replace_word = ['เพศสัมพันธุ์','เพศสัมพัน','เพสสัมพัน']
    list_words = ['สวัสดี','ขอบคุณ','อยู่แล้ว' , 'สัปดาห์' ,'หนูก้อรีบ' ,'ชอบเป้น' ,'นิดนึง' ,'ตุลาเลย' , 'เหมือน' , 'สอบถาม', 'อะครับ','คะอยาก','เสี่ยง','ปริมาณ','ต่อเนื่อง','เปื้อน','กางเกง',
                 'สุดท้าย','เรื่อง','รึป่าว','มากน้อย','ชั่วโมง','ประมาณ','บริเวณ','เชี่ยวชาญ','พยาบาล','ประมาน','นิดเดียว','สม่ำเสมอ','กคประจำ','วี่แวว','เที่ยง','คะแล้ว','ลักษณะ',
                 'อาทิตย์','จันทร์','อังคาร','พุธ','พฤหัส','ศุกร์','เสาร์','คะดิฉัน','สองสาม','รุ้ว่า','ปากช่อง','รึเปล่า','สังเกตุ','ทั้งๆที่','เป็นอยู่','มิถุนา','ได้ยิน','แน่นอน','เครื่อง','ตัดสินใจ','อวัยวะ','สมมุติ','เตรียม','ล่วงหน้า','เรียบร้อย','คับคือ',
                 'เมษายน','ธันวาคม','พฤษภาคม','พฤศจิกายน','มกราคม','กุมภาพันธ์','มีนาคม','มิถุนายน','สิงหาคม','กรกฎาคม','ตุลาคม','กันยายน',
                 'จังหวัด', 'เลื่อน','ปรึกษา', 'เกี่ยว','ต้องการ','รู้สึก','สมบูรณ์','มั่นใจ', 'เนื่อง', 'สำหรับ', 'คะพอดี','รุ้สึก','หาสจาก','ร่างกาย','ห้องน้ำ']
    #list_words = []
    result = []
    for token in word_tokenize(documents, engine='newmm'): 
        token = token.strip()
        if token not in stopwords and len(token) >5 and token not in list_words:
            # token = correct(token)
            # token = token.replace('สัมพันธุ์','สัมพันธ์')
            # token = token.replace('สัมพัน','สัมพันธ์')
            # token = token.replace('สัมพันธ์ธ์','สัมพันธ์')
            # token = token.replace('เพส','สัมพันธ์')
            # token = correct(str(token))
            result.append(token)
    return result


def func(documents):
    res = documents['headline_text'].map(preprocess)
    return res

def cleanText(documents):
    documents['headline_text'] = documents['headline_text'].str.replace('|',' ')
    documents['headline_text'] = documents['headline_text'].str.replace('\\',' ')
    documents['headline_text'] = documents['headline_text'].str.replace('\/',' ')
    documents['headline_text'] = documents['headline_text'].str.replace('.',' ')
    documents['headline_text'] = documents['headline_text'].str.replace('_','')
    documents['headline_text'] = documents['headline_text'].str.replace('\d+', '')
    documents['headline_text'] = documents['headline_text'].str.replace('-',' ')
    documents['headline_text'] = documents['headline_text'].str.replace('+',' ')
    documents['headline_text'] = documents['headline_text'].str.replace('+',' ')
    pattern = '|'.join(['&', '%', ';', '='])
    documents['headline_text'] = documents['headline_text'].str.replace(pattern, ' ')
    # documents = documents.loc[~documents['headline_text'].str.contains('img')]
    documents.headline_text.replace({r'[^ก-๙]':''}, regex=True, inplace=True)
    documents['headline_text'] = documents['headline_text'].str.replace(':',' ')
    documents['headline_text'] = documents['headline_text'].str.replace('#',' ')
    documents['headline_text'] = documents['headline_text'].str.strip()
    documents = documents.loc[documents['headline_text']!= ""]
    documents = documents.dropna().reset_index(drop=True)
    return documents


In [23]:
# documents = dataset.copy()
# documents = documents.rename(columns={'question':'headline_text'})

# # concatenate all string of same user to be headline_text
# documents = documents.reset_index()
# print(documents.shape[0])
# documents.head()
dataset = dataset.iloc[:,:]
documents = dataset.copy()
documents = documents.rename(columns={'question':'headline_text'})

# # concatenate all string of same user to be headline_text
# # documents = documents.agg(lambda col: ' '.join(col)).reset_index()
# documents = documents.groupby(['headline_text'], as_index = False).agg({'headline_text': lambda x: ' '.join(x.astype(str))})
# print(documents.shape[0])
documents = documents.groupby('user')['headline_text'].agg(lambda col: ' '.join(col)).reset_index()
print(documents.shape[0])
documents.head()




8251


,user,headline_text
0,\nHonestDocs User \n,ตั้งครรภ์35week่ท้องแข็งบ่อยมากจะมีอันตรายกับเ...
1,\nHonestDocs User 1000031\n,รบกวนสอบถามหน่อยค่ะ มีเพศสัมพันธ์ประมาณวันที่ ...
2,\nHonestDocs User 1000061\n,ถ้าผมช่วยตัวเองเสร็จก่อนหน้าแล้วผมใช้ผ้าเช็ดน้...
3,\nHonestDocs User 1000074\n,ถ้ายังไม่พร้อมมีลูกทำแท้งผิดกฎหมายไหมครับ ปลอา...
4,\nHonestDocs User 1000159\n,สวัสดีค่ะคุณหมอ พอดีว่าปกติเป็นคนทานยาคุมพรีม2...


In [24]:
documents = parallelize(documents,cleanText)
documents.head()


,user,headline_text
0,\nHonestDocs User \n,ตั้งครรภ์่ท้องแข็งบ่อยมากจะมีอันตรายกับเด็กในค...
1,\nHonestDocs User 1000031\n,รบกวนสอบถามหน่อยค่ะมีเพศสัมพันธ์ประมาณวันที่แล...
2,\nHonestDocs User 1000061\n,ถ้าผมช่วยตัวเองเสร็จก่อนหน้าแล้วผมใช้ผ้าเช็ดน้...
3,\nHonestDocs User 1000074\n,ถ้ายังไม่พร้อมมีลูกทำแท้งผิดกฎหมายไหมครับปลอาย...
4,\nHonestDocs User 1000159\n,สวัสดีค่ะคุณหมอพอดีว่าปกติเป็นคนทานยาคุมพรีมเม...


In [25]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
from pythainlp.tokenize import word_tokenize


allLen = documents.shape[0]
progress = 0
# Print iterations progress
def printProgressBar (iteration, total,mode = 0,current_percent = 0, prefix = '', suffix = '', decimals = 1, length = 100, fill = ' '):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        mode        - Optional  : Mode of output (Int : (dafault 0 : grahical progress bar , 1 : standard text))
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    if mode == 0 :
        filledLength = int(length * iteration // total)
        bar = fill * filledLength + '-' * (length - filledLength)
        print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
        # Print New Line on Complete
        if iteration == total:
            print()
    elif mode == 1:
        if current_percent != round(float(percent),1):
            current_percent = round(float(percent),1)
            print(percent+"%")
        return current_percent
    return 0

In [26]:
!pip install pytesseract

In [27]:
# stemmer = PorterStemmer()


In [28]:
processed_docs = documents['headline_text'].map(preprocess)

In [29]:
# start_time = time.time()
# for token in processed_docs: 
#   arr= []
#   for word in token: 
#       token = word
#       word = correct(word)
#       arr.append(word)
# print(arr)


# print("--- %s seconds ---" % (time.time() - start_time))

In [30]:
# processed_docs = correctword(processed_docs)

In [31]:
processed_docs

0       [ตั้งครรภ์, ท้องแข็ง, อันตราย, ตั้งครรภ์, ท้อง...
1       [มีเพศสัมพันธ์, วันที่, วันที่, การป้องกัน, ตอ...
2       [ช่วยตัวเอง, น้ำอสุจิ, ผู้หญิง, เป็นไร, ข้างนอ...
3                                     [ทำแท้ง, ผิดกฎหมาย]
4       [คุณหมอ, มาโดยตลอด, ตอนนี้, ประจำเดือน, คุณหมอ...
                              ...                        
4118        [เข้าเดือน, ป้องกัน, หน้าอก, น้ำหนัก, ตอนนี้]
4119    [ครั้งสุดท้าย, วันที่, วันที่, วันที่, ประจำเด...
4120                                         [ประจำเดือน]
4121    [ฝากครรภ์, รังสิต, ตั้งครรภ์, ตอบสนอง, ค่าใช้จ...
4122               [ทำหมัน, โรงพยาบาล, ชลบุรี, สิทธิบัตร]
Name: headline_text, Length: 8248, dtype: object

In [32]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 กรรมพันธุ์
1 กระดูก
2 กระตุก
3 กระเพาะ
4 กลับบ้าน
5 กลับมา
6 กลับหัว
7 กลางคืน
8 กลางดึก
9 กลางปี
10 กลายเป็น


In [33]:
dictionary

In [34]:
from gensim import corpora, models

In [35]:
# # ตัดคำฟุ่มเฟือยออก โดยลบคำที่ปรากฏบ่อยเกิน 50% และให้เหลือไปถึง1พันคำ
# dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=1000)  #reduce to 1K word


# # Gensim doc2bow
# # For each document we create a dictionary reporting how many
# # words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.
# bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]


# tfidf = models.TfidfModel(bow_corpus)
# corpus_tfidf = tfidf[bow_corpus]
# from pprint import pprint
# for doc in corpus_tfidf:
#     pprint(doc)
#     break

# # run LDA แบ่งคำออกเป็น9กลุ่ม โดยใช้ cpu ทุกตัว (serverที่เราใช้มี32cpu)
# #worker = cpu core amount -1
# lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=31)

# # For each topic, we will explore the words occuring in that topic and its relative weight.
# for idx, topic in lda_model.print_topics(-1):
#     print('Topic: {} \nWords: {}'.format(idx, topic))

In [36]:
# unseen_document = 'ตั้งครรภ์5เดือนแล้วคะมีตกขาวสีน้ำตาลขุ่นบ้างทีก็แดงเข้มแห้งๆไม่มีกลิ่นไม่มีอาการผิดปกติอยากทราบว่าอันตรายไหมค้ะเกิดจากสาเหตุอะไระ'
# bow_vector = dictionary.doc2bow(preprocess(unseen_document))
# for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
#     print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 15)))

In [37]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=1000)

# Dictionary for each document
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
# print(bow_corpus[4310])

# Applying TF-IDF
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

# Train corpus with LDA
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=7, id2word=dictionary,
                                       passes=2, workers=31)

# for idx, topic in lda_model.print_topics(-1):
#     print('Train corpus with LDA \n'+'Topic: {} \nWords: {}'.format(idx, topic))

# LDA with TF-IDF
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=7, id2word=dictionary,
                                             passes=2, workers=31)
# for idx, topic in lda_model_tfidf.print_topics(-1):
#     print('LDA with TF-IDF \n'+'Topic: {} Word: {}'.format(idx, topic))

# # Performance evaluation of LDA with Bag of Words
# for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
#     print('Performance evaluation of LDA with Bag of Words \n'+"\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

# # Performance Evaluation of LDA with TF-IDF
# for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
#     print('Performance Evaluation of LDA with TF-IDF \n'+"\nScore: {} \t \nTopic: {}".format(score,lda_model_tfidf.print_topic(index, 10)))

# # Testing
# unseen_document = 'ตั้งครรภ์5เดือนแล้วคะมีตกขาวสีน้ำตาลขุ่นบ้างทีก็แดงเข้มแห้งๆไม่มีกลิ่นไม่มีอาการผิดปกติอยากทราบว่าอันตรายไหมค้ะเกิดจากสาเหตุอะไระ'
# print(unseen_document)
# bow_vector = dictionary.doc2bow(preprocess(unseen_document))
# for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
#     print("\nScore: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

[(0, 0.008662700125368166),
 (1, 0.00625806811313018),
 (2, 0.016726727295634285),
 (3, 0.008535520488197305),
 (4, 0.006757583839494905),
 (5, 0.014867549257207567),
 (6, 0.015885749877259676),
 (7, 0.024484311198403275),
 (8, 0.009222498365166032),
 (9, 0.008662700125368166),
 (10, 0.08070054575800702),
 (11, 0.015885749877259676),
 (12, 0.007902827398679821),
 (13, 0.020438240264403766),
 (14, 0.006021806038514317),
 (15, 0.014605112904462451),
 (16, 0.07787159514840945),
 (17, 0.008730695816441227),
 (18, 0.015044772458885063),
 (19, 0.018079108873852046),
 (20, 0.05681518016204181),
 (21, 0.00859772748350275),
 (22, 0.03716887314301892),
 (23, 0.06532207191625088),
 (24, 0.34390909934010994),
 (25, 0.027752012496921163),
 (26, 0.031303524054224104),
 (27, 0.009039554436926023),
 (28, 0.02840331791377183),
 (29, 0.008161437066134425),
 (30, 0.028860550415148716),
 (31, 0.016726727295634285),
 (32, 0.017753964360414376),
 (33, 0.00912816603776477),
 (34, 0.013306272579836555),
 (35,

In [38]:
print('Train corpus with LDA')
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}\n'.format(idx, topic))

print('LDA with TF-IDF')
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}\n'.format(idx, topic))

print('Performance evaluation of LDA with Bag of Words')
# Performance evaluation of LDA with Bag of Words
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("Score: {}\t \nTopic: {}\n".format(score, lda_model.print_topic(index, 10)))

print('Performance Evaluation of LDA with TF-IDF')
# Performance Evaluation of LDA with TF-IDF
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("Score: {} \t \nTopic: {}\n".format(score,lda_model_tfidf.print_topic(index, 10)))

# Testing
unseen_document = 'ตั้งครรภ์5เดือนแล้วคะมีตกขาวสีน้ำตาลขุ่นบ้างทีก็แดงเข้มแห้งๆไม่มีกลิ่นไม่มีอาการผิดปกติอยากทราบว่าอันตรายไหมค้ะเกิดจากสาเหตุอะไระ'
print(unseen_document)
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}\n".format(score, lda_model_tfidf.print_topic(index, 10)))

Train corpus with LDA
Topic: 0 
Words: 0.287*"ประจำเดือน" + 0.052*"ตั้งครรภ์" + 0.044*"ตอนนี้" + 0.038*"วันที่" + 0.030*"มีเพศสัมพันธ์" + 0.028*"ได้การ" + 0.028*"ผู้เชี่ยวชาญ" + 0.023*"แบบนี้" + 0.016*"ฉุกเฉิน" + 0.015*"มีโอกาส"

Topic: 1 
Words: 0.107*"คุณหมอ" + 0.044*"ยี่ห้อ" + 0.040*"ตอนนี้" + 0.039*"ป้องกัน" + 0.036*"มีเพศสัมพันธ์" + 0.030*"ฉุกเฉิน" + 0.030*"ประจำเดือน" + 0.030*"ตั้งครรภ์" + 0.026*"ตรงเวลา" + 0.023*"เป็นไร"

Topic: 2 
Words: 0.134*"ประจำเดือน" + 0.098*"มีเพศสัมพันธ์" + 0.090*"ฉุกเฉิน" + 0.041*"มีโอกาส" + 0.032*"ถุงยาง" + 0.029*"แบบนี้" + 0.028*"สอดใส่" + 0.027*"ทำหมัน" + 0.023*"ตอนนี้" + 0.016*"คุณหมอ"

Topic: 3 
Words: 0.113*"มีโอกาส" + 0.078*"ตั้งครรภ์" + 0.053*"มีเพศสัมพันธ์" + 0.041*"ประจำเดือน" + 0.035*"อวัยวะเพศ" + 0.032*"ถุงยาง" + 0.027*"ตอนนี้" + 0.027*"หล่อลื่น" + 0.020*"ป้องกัน" + 0.019*"สอดใส่"

Topic: 4 
Words: 0.068*"ฉุกเฉิน" + 0.067*"แบบนี้" + 0.030*"ตอนนี้" + 0.028*"ครับผม" + 0.028*"ป้องกัน" + 0.025*"ตั้งครรภ์" + 0.024*"ถุงยาง" + 0.024*"แล้วก็" + 0.0

In [39]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[bow_corpus]

[(0,
  '0.287*"ประจำเดือน" + 0.052*"ตั้งครรภ์" + 0.044*"ตอนนี้" + 0.038*"วันที่" + '
  '0.030*"มีเพศสัมพันธ์" + 0.028*"ได้การ" + 0.028*"ผู้เชี่ยวชาญ" + '
  '0.023*"แบบนี้" + 0.016*"ฉุกเฉิน" + 0.015*"มีโอกาส"'),
 (1,
  '0.107*"คุณหมอ" + 0.044*"ยี่ห้อ" + 0.040*"ตอนนี้" + 0.039*"ป้องกัน" + '
  '0.036*"มีเพศสัมพันธ์" + 0.030*"ฉุกเฉิน" + 0.030*"ประจำเดือน" + '
  '0.030*"ตั้งครรภ์" + 0.026*"ตรงเวลา" + 0.023*"เป็นไร"'),
 (2,
  '0.134*"ประจำเดือน" + 0.098*"มีเพศสัมพันธ์" + 0.090*"ฉุกเฉิน" + '
  '0.041*"มีโอกาส" + 0.032*"ถุงยาง" + 0.029*"แบบนี้" + 0.028*"สอดใส่" + '
  '0.027*"ทำหมัน" + 0.023*"ตอนนี้" + 0.016*"คุณหมอ"'),
 (3,
  '0.113*"มีโอกาส" + 0.078*"ตั้งครรภ์" + 0.053*"มีเพศสัมพันธ์" + '
  '0.041*"ประจำเดือน" + 0.035*"อวัยวะเพศ" + 0.032*"ถุงยาง" + 0.027*"ตอนนี้" + '
  '0.027*"หล่อลื่น" + 0.020*"ป้องกัน" + 0.019*"สอดใส่"'),
 (4,
  '0.068*"ฉุกเฉิน" + 0.067*"แบบนี้" + 0.030*"ตอนนี้" + 0.028*"ครับผม" + '
  '0.028*"ป้องกัน" + 0.025*"ตั้งครรภ์" + 0.024*"ถุงยาง" + 0.024*"แล้วก็" + '
  '0.022*"ยาเม็

**============================================================================**

In [40]:
!pip3 install spacy
!python3 -m spacy download en #Language model
!pip3 install gensim # For topic modeling
!pip install pyLDAvis # For visualizing topic models


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [41]:
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [42]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(bow_corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt


Perplexity:  -4.617117388171485

Coherence Score:  0.4137657751116661


In [43]:
# Compute Perplexity
print('\nPerplexity: ', lda_model_tfidf.log_perplexity(bow_corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model_tfidf, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt


Perplexity:  -4.8165688779005675

Coherence Score:  0.40687816458199616


Output:
- Lower the perplexity better the model.
- Higher the topic coherence, the topic is more human interpretable.

In [44]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
vis




PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.085249  0.040253       1        1  26.543161
0      0.045732  0.068572       2        1  17.410048
6     -0.121511  0.051660       3        1  15.931723
3     -0.005831 -0.044822       4        1  13.933301
2      0.107899 -0.060919       5        1  10.337750
1     -0.036482  0.049582       6        1   8.454673
4     -0.075057 -0.104326       7        1   7.389343, topic_info=           Term         Freq        Total Category  logprob  loglift
107      วันที่  3370.000000  3370.000000  Default  30.0000  30.0000
74   ประจำเดือน  4623.000000  4623.000000  Default  29.0000  29.0000
28      ฉุกเฉิน  1300.000000  1300.000000  Default  28.0000  28.0000
97      มีโอกาส  1389.000000  1389.000000  Default  27.0000  27.0000
23       คุณหมอ  1289.000000  1289.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
145      เข้าไป    33.877346   363.944859   Topic7  -4.5324   0.2309
97      มีโอกาส    47.972709  1389.651194   Topic7  -4.1845  -0.7610
25    คุมกำเนิด    27.834398   219.813424   Topic7  -4.7289   0.5386
23       คุณหมอ    41.516523  1289.611965   Topic7  -4.3291  -0.8309
74   ประจำเดือน    48.710964  4623.958666   Topic7  -4.1693  -1.9480

[410 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.183017  กลับบ้าน
0         2  0.036603  กลับบ้าน
0         3  0.146414  กลับบ้าน
0         4  0.073207  กลับบ้าน
0         5  0.109810  กลับบ้าน
...     ...       ...       ...
186       2  0.066107   ไม่ต้อง
186       3  0.082633   ไม่ต้อง
186       4  0.066107   ไม่ต้อง
186       5  0.347061   ไม่ต้อง
186       6  0.181794   ไม่ต้อง

[1429 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 7, 4, 3, 2, 5])

In [45]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.022816  0.014449       1        1  27.292072
4     -0.035942 -0.046291       2        1  17.426774
1     -0.079718 -0.027407       3        1  15.234543
2     -0.016274  0.069429       4        1  12.855353
5     -0.012470  0.031026       5        1  10.161418
3      0.071792 -0.081653       6        1   8.552976
6      0.095427  0.040448       7        1   8.476865, topic_info=            Term         Freq        Total Category  logprob  loglift
213       ถุงยาง   755.000000   755.000000  Default  30.0000  30.0000
107       วันที่  2650.000000  2650.000000  Default  29.0000  29.0000
137      อันตราย   208.000000   208.000000  Default  28.0000  28.0000
40        ตอนนี้  1222.000000  1222.000000  Default  27.0000  27.0000
74    ประจำเดือน  2686.000000  2686.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
28       ฉุกเฉิน    51.608307  1081.419164   Topic7  -4.2488  -0.5745
97       มีโอกาส    53.835302  1293.583517   Topic7  -4.2066  -0.7114
60        ที่สอง    28.207480   190.768008   Topic7  -4.8529   0.5564
129  หลังจากนั้น    31.325611   384.696671   Topic7  -4.7480  -0.0402
80       ป้องกัน    26.202123   723.456672   Topic7  -4.9266  -0.8504

[387 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.120747  กลับบ้าน
0         2  0.090560  กลับบ้าน
0         3  0.060373  กลับบ้าน
0         4  0.150933  กลับบ้าน
0         6  0.422614  กลับบ้าน
...     ...       ...       ...
186       2  0.022785   ไม่ต้อง
186       3  0.125316   ไม่ต้อง
186       4  0.056962   ไม่ต้อง
186       5  0.535443   ไม่ต้อง
186       7  0.011392   ไม่ต้อง

[1478 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 2, 3, 6, 4, 7])

In [46]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 78.1636323928833 seconds ---


bubble แต่ละอันคือ topic <br>
bubble ที่ใหญ่จะมีความแพร่หลายและโดดเด่นมาก <br>

รูปแบบหัวข้อที่ดีจะเป็นหัวข้อที่ค่อนข้างใหญ่ซึ่งกระจัดกระจายอยู่ในควอดแดรนต์ที่แตกต่างกันแทนที่จะกระจุกอยู่ในจตุภาคเดียว  <br>

- The model with too many topics will have many overlaps, small sized bubbles clustered in one region of chart.
- If you move the cursor the different bubbles you can see different keywords associated with topics.